Data size < 1MB, hence loading the entire content into memeory is feasible

In [1]:
from string import punctuation

In [8]:
import numpy as np

In the block below, I collect and count the number of occurances of each word in the corpurs. 

<b>In the below block, in the line - "for line in open(....)", please replace 'training.data' with the relative path of the training file</b>

In [2]:
allWords=dict() # dict stores the count of occurances of every word in the corpus
sentences=[]    # Tokenized sentences 
label1=[]       # Collecting labels
label2=[]       # Collecting labels
lab1Count=1     
lab2Count=1
label1dict=dict()  # Assigning numerical values to labels
label2dict=dict()  # Assigning numerical values to labels

for line in open('training.data','r',encoding="ISO-8859-1"):
    line=line.strip().split(' ')
    labels=line[0].split(':')
    if(len(labels)!=2 or ' ' in line[0]):
        print('Formatting asssumption wrong')
        break
    sentence=[word for word in line[1:] if word not in punctuation]
    sentences.append(list(sentence))
    for word in list(sentence):
        if(allWords.get(word,0)==0):
            allWords[word]=1
        else:
            allWords[word]+=1
    if(not(label1dict.get(labels[0],0))):
        label1dict[labels[0]]=lab1Count
        lab1Count+=1
    if(not(label2dict.get(labels[1],0))):
        label2dict[labels[1]]=lab2Count
        lab2Count+=1
    label1.append(label1dict[labels[0]])
    label2.append(label2dict[labels[1]])

In the block below, I collect the 500 most frequently occuring words

In [3]:
popularTokens=[x[0] for x in sorted(allWords.items(),key=lambda x:x[1],reverse=True)[:500]]

The following function converts each sentence into a 500 long vector representation.

In [6]:
def wordVec(sent):
    vec=[]
    for word in popularTokens:
        if word in sent:
            vec.append(1)
        else:
            vec.append(0)
    return vec

vectors is a list that contains the 500-long vector representations

In [5]:
vectors=[wordVec(eachSentence) for eachSentence in sentences]
    

In [15]:
#Training set
trainVectors=np.array(vectors[:int(0.6*len(vectors))])
trainLabels1=np.array(label1[:int(0.6*len(vectors))])
trainLabels2=np.array(label2[:int(0.6*len(vectors))])


In [18]:
#Validation Set
validVectors=np.array(vectors[int(0.6*len(vectors)):int(0.8*len(vectors))])
validLabels1=np.array(label1[int(0.6*len(vectors)):int(0.8*len(vectors))])
validLabels2=np.array(label2[int(0.6*len(vectors)):int(0.8*len(vectors))])

In [19]:
#test Set
testVectors=np.array(vectors[int(0.8*len(vectors)):])
testLabels1=np.array(label1[int(0.8*len(vectors)):])
testLabels2=np.array(label2[int(0.8*len(vectors)):])


I am training two classifiers, one for the primary label that occurs before the ':' and the other is for the 
secondary label.
Using soft margin SVM with a varying cost parameters before fixing on one.

In [11]:
from sklearn import svm

In [20]:
for c in [100,1000,10**5]:
    clf=svm.SVC(C=c, kernel='rbf', gamma='auto', tol=0.001, class_weight='balanced', max_iter=-1,\
                decision_function_shape='ovr')
    clf.fit(trainVectors,trainLabels1)
    preds=clf.predict(validVectors)
    accu=sum([1 if i==j else 0 for i,j in zip(preds,validLabels1)])/len(validLabels1)
    print(accu)


0.8809061488673139
0.9080906148867314
0.9122977346278317


For the primary label, I decide to use an SVM with C=10**5

Reporting the test accuracy

In [45]:
clf=svm.SVC(C=10**5, kernel='rbf', gamma='auto', tol=0.001, class_weight='balanced', max_iter=-1,\
                decision_function_shape='ovr')
clf.fit(np.vstack([trainVectors,validVectors]),np.hstack([trainLabels1,validLabels1]))
preds=clf.predict(testVectors)
accu=sum([1 if i==j else 0 for i,j in zip(preds,testLabels1)])/len(testLabels1)
print('Test accuracy obtained for primary label is ',accu)

0.8670333225493367


In [51]:
import random

Lets see if it is actually working

In [56]:
label1dictReverse=dict((i,j) for i,j in zip(list(label1dict.values()),list(label1dict.keys())))

In [59]:
for i in range(10):
    randNum=random.randint(0,len(vectors))
    print(sentences[randNum])
    print('predicted label is ',label1dictReverse[clf.predict([vectors[randNum]])[0]])
    print('actual label is ',label1dictReverse[label1[randNum]])

['What', 'developed', 'a', 'crack', 'in', '1835', 'while', 'tolling', 'the', 'death', 'of', 'U.S.', 'Chief', 'Justice', 'John', 'Marshall']
predicted label is  ENTY
actual label is  ENTY
['What', 'was', 'the', 'name', 'of', 'the', 'American', 'ship', 'sunk', 'by', 'a', 'mine', 'in', 'Havana', 'harbor', 'causing', 'the', 'Spanish-American', 'war']
predicted label is  ENTY
actual label is  ENTY
['Who', 'is', 'Prince', 'Naseem', 'Hamed']
predicted label is  HUM
actual label is  HUM
['What', 'is', 'the', 'name', 'of', 'the', 'vaccine', 'for', 'chicken', 'pox']
predicted label is  ENTY
actual label is  ENTY
['How', 'do', 'I', 'start', 'a', 'bank']
predicted label is  DESC
actual label is  DESC
['What', 'are', 'the', '7', 'articles', 'of', 'the', 'constitution']
predicted label is  ENTY
actual label is  ENTY
['Which', 'of', 'the', 'following', 'was', 'Rhodes', 'Scholar']
predicted label is  HUM
actual label is  HUM
['What', 'is', 'the', 'temperature', 'for', 'baking', 'Peachy', 'Oat', 'Muffi

Looks like it actually is

Hence, 1 Gram results in a test accuracy of 86.7%.

Repeating the process for the secondary label

In [36]:
for c in [100,1000,10**5]:
    clf=svm.SVC(C=c, kernel='rbf', gamma='auto', tol=0.001, class_weight='balanced', max_iter=-1,\
                decision_function_shape='ovr')
    clf.fit(trainVectors,trainLabels2)
    preds=clf.predict(validVectors)
    accu=sum([1 if i==j else 0 for i,j in zip(preds,validLabels2)])/len(validLabels2)
    print(accu)

0.7964401294498382
0.8766990291262136
0.8919093851132686


I decide to use c=10**5 to train svm for the secondary label as well

In [37]:
clf=svm.SVC(C=10**5, kernel='rbf', gamma='auto', tol=0.001, class_weight='balanced', max_iter=-1,\
                decision_function_shape='ovr')
clf.fit(np.vstack([trainVectors,validVectors]),np.hstack([trainLabels2,validLabels2]))
preds=clf.predict(testVectors)
accu=sum([1 if i==j else 0 for i,j in zip(preds,testLabels2)])/len(testLabels2)
print("Test accuracy obtained for secondary label is",accu)

0.8052410223228729


This accuracy greatly surprises me. I believe fastText, which I plan to use next, is the best in class and results in ~89% accuracy. 

Experimenting with ensembles results in similar numbers but with much greater training time.

In [13]:
from sklearn.ensemble import RandomForestClassifier

In [14]:
clf=RandomForestClassifier(n_estimators=50, criterion='gini', max_depth=500, min_samples_split=2, \
                            min_weight_fraction_leaf=0.0, max_features='auto', max_leaf_nodes=None, \
                            n_jobs=1, random_state=None, verbose=0, warm_start=False, class_weight='balanced')


clf.fit(trainVectors,trainLabels1)

preds=clf.predict(testVectors)
accu=sum([1 if i==j else 0 for i,j in zip(preds,testLabels1)])/len(testLabels1)
print(accu)

0.8657392429634423
